### Non-personalized RS

In [3]:
import pandas as pd
import numpy as np

from scipy.sparse import csr_matrix
%matplotlib inline
import matplotlib.pyplot as plt

In [4]:
JSON_DATA_PATH = "data/reviews.json"
N = 10

In [5]:
import json

def iter_json_data(path):
    with open(path) as f:
        for line in f:
            data = json.loads(line)
            yield data
            
def get_data_frame():
    uid_to_id = {}
    iid_to_id = {}
    
    cols = ["uid", "iid", "review", "rating", "dt", "helpful", "summary"]
    rows = []
    for d in iter_json_data(JSON_DATA_PATH):
        uid = uid_to_id.setdefault(d["reviewerID"], len(uid_to_id))
        iid = iid_to_id.setdefault(d["asin"], len(iid_to_id))
        review = d["reviewText"]
        rating = float(d["overall"])
        dt = int(d["unixReviewTime"])
        helpful = 0 if d["helpful"][1] == 0 else d["helpful"][0]/d["helpful"][1]
        summary = d["summary"]
        rows.append((uid, iid, review, rating, dt, helpful, summary))
        
    return pd.DataFrame(rows, columns=cols)

In [6]:
df = get_data_frame()
df.head()

,uid,iid,review,rating,dt,helpful,summary
0,0,0,Installing the game was a struggle (because of...,1.0,1341792000,0.666667,Pay to unlock content? I don't think so.
1,1,0,If you like rally cars get this game you will ...,4.0,1372550400,0.000000,Good rally game
2,2,0,1st shipment received a book instead of the ga...,1.0,1403913600,0.000000,Wrong key
3,3,0,"I got this version instead of the PS3 version,...",3.0,1315958400,0.700000,"awesome game, if it did not crash frequently !!"
4,4,0,I had Dirt 2 on Xbox 360 and it was an okay ga...,4.0,1308009600,1.000000,DIRT 3


In [7]:
def split_df_by_dt(df, p=0.8):
    """Функция разбивает df на тестовую и тренировочную выборки по времени 
    публикации отзывов (значение времени в поле dt)
    
    :param p: персентиль значений dt, которые образуют тренировочную выборку. Например p=0.8 означает, что в 
    тренировочной части будут отзывы, соответствующие первым 80% временного интервала 
    :return: два pd.DataFrame объекта
    """
    border_dt = df.dt.quantile(p)
    print("Min=%s, border=%s, max=%s" % (df.dt.min(), border_dt, df.dt.max()))
    training_df, test_df  = df[df.dt <= border_dt], df[df.dt > border_dt]
    print("Размер до очистки:", training_df.shape, test_df.shape)
    # удаляем из тестовых данных строки, соответствующие пользователям или объектам, 
    # которых нет в тренировочных данных 
    # (пользователи - избегаем проблем для персональных систем, объекты - для всех)
    test_df = test_df[test_df.uid.isin(training_df.uid) & test_df.iid.isin(training_df.iid)]
    print("Размер после очистки:", training_df.shape, test_df.shape)
    return training_df, test_df

In [8]:
def clean_df(df, min_review_per_uid, min_review_per_iid):
    """Функция удаляет из df строки, соответствующие пользователям и объектам, 
    у которых меньше min_review_per_uid и min_review_per_iid отзывов соответственно
    """
    _df = df.copy()
    while True:
        review_per_uid = _df.groupby("uid").review.count()
        bad_uids = review_per_uid[review_per_uid < min_review_per_uid].index
    
        review_per_iid = _df.groupby("iid").review.count()
        bad_iids = review_per_iid[review_per_iid < min_review_per_iid].index
        
        if bad_uids.shape[0] > 0 or bad_iids.shape[0] > 0:
            _df = _df[(~_df.uid.isin(bad_uids)) & (~_df.iid.isin(bad_iids))]
        else:
            break
    return _df

In [9]:
def hit_ratio(recs_dict, test_dict):
    """Функция считает метрику hit-ration для двух словарей
    :recs_dict: словарь рекомендаций типа {uid: {iid: score, ...}, ...}
    :test_dict: тестовый словарь типа {uid: {iid: score, ...}, ...}
    """
    hits = 0
    for uid in test_dict:
#         print(test_dict[uid].keys())
#         print(recs_dict.get(uid, {}))
        if set(test_dict[uid].keys()).intersection(recs_dict.get(uid, {})):
            hits += 1
    return hits / len(test_dict)

In [10]:
training_df, test_df = split_df_by_dt(df)
del df

Min=939859200, border=1377129600.0, max=1405987200
Размер до очистки: (185427, 7) (46353, 7)
Размер после очистки: (185427, 7) (19174, 7)


In [11]:
def get_test_dict(test_df):
    """Функция, конвертирующая тестовый df в словарь
    """
    test_dict = {}
    for t in test_df.itertuples():
        test_dict.setdefault(t.uid, {})
        test_dict[t.uid][t.iid] = t.rating
    return test_dict

test_dict = get_test_dict(test_df)

In [12]:
class BasicRecommender(object):
    def __init__(self):
        pass
    
    def get_recs(self, uid, top):
        """Строит рекомендации для пользователя uid
        :return: словарь типа {iid: score, ...}
        """
        return {}
    
    def get_batch_recs(self, uids, top):
        """Строит рекомендации для нескольких пользователей uids
        :return: словарь типа {uid: {iid: score, ...}, ...}
        """
        return {uid: self.get_recs(uid, top) for uid in uids}
    
class NonPersRecommender(BasicRecommender):
    def __init__(self, df):
        super(NonPersRecommender, self).__init__()
        self.recs = self._prepare_recs(df)
        
    def _prepare_recs(self, df):
        return pd.Series([])
    
    def get_recs(self, uid, top):
        return self.recs[:top].to_dict()
    
    def get_batch_recs(self, uids, top):
        non_pers_recs = self.get_recs(None, top)
        return {uid: non_pers_recs for uid in uids}
    
class MostReviewedRS(NonPersRecommender):
    def _prepare_recs(self, df):
        # считаем количество отзывов для каждого объекта (pandas сортирует их по убыванию)

        return df.iid.value_counts()

In [13]:
# посмотрим на даты ревью
training_df['date'] = pd.to_datetime(training_df['dt'],unit='s')
print('first date ' + str(training_df['date'].min()) + '   last date ' + str(training_df['date'].max()))

first date 1999-10-14 00:00:00   last date 2013-08-22 00:00:00


### Рекомендации на основе среднего рейтинга игр и даты первого отзыва о ней

In [227]:
class MeanRatingRS(NonPersRecommender):
    
    def __init__(self, df, month, year):
        
        self.y = year
        self.m = month
        self.recs = self._prepare_recs(df)
        
    
    def _prepare_recs(self, df):
        df['date'] = pd.to_datetime(df['dt'],unit='s')
        # посчитаем средний рейтинг, количество оценок и дату самого раннего отзыва
        d = df.groupby('iid').agg({"rating": {"mean", "count"}, "date" : {"min"}}).reset_index()
        d.columns = d.columns.droplevel(0)
        d.rename(columns={'min': 'date'}, inplace=True)
        d['date'] = pd.to_datetime(d['date'],unit='s')
        #         
        d = d[(d['date'].dt.year > self.y) & (d['date'].dt.month > self.m)] \
            .sort_values(['count'], ascending = False)
            
        d.rename(columns={'': 'iid'}, inplace=True)
        d.set_index('iid', inplace=True)
        d = d[(d['mean'] < 5) & (d['mean'] > 4)]
        return d['count'].T

In [228]:
test_dict = get_test_dict(test_df)

In [229]:
rs_most_reviewed = MostReviewedRS(training_df)
recs_dict = rs_most_reviewed.get_batch_recs(test_df.uid, N)
hit_ratio(recs_dict, test_dict )

0.03961848862802641

In [230]:
# ищем лучшее соотношение за какой промежуток времени считать 
params = {'month' : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], 
          'year' : [1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013]}

In [231]:
def search_best_params(params):
    scores = []
    for year in params['year']:
        for month in params['month']:
            rs_mean_rating = MeanRatingRS(training_df, month, year)
            recs_dict = rs_mean_rating.get_batch_recs(test_df.uid, N)
            scores.append([hit_ratio(recs_dict, test_dict), month, year])
    return sorted(scores)[::-1][0]

In [232]:
best_param = search_best_params(params)

In [233]:
best_param

[0.07879677182685253, 2, 2012]

In [234]:
rs_mean_rating = MeanRatingRS(training_df, best_param[1], best_param[2])
recs_dict = rs_mean_rating.get_batch_recs(test_df.uid, N)
hit_ratio(recs_dict, test_dict)

0.07879677182685253